In [1]:
%cd /kaggle/working

/kaggle/working


In [2]:
import pickle
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns
from hydra import compose, initialize
from omegaconf import OmegaConf

from utils.metric import score

In [3]:
def hide_submission(sub_df):
    hide_sub_df = sub_df.clone().with_columns(
        [pl.lit(0.0).alias(f"ptend_q0002_{i}") for i in range(12, 22)]
    )
    return hide_sub_df

In [4]:
base_sub_path = "gs://kaggle-leap/kami/20240610_ensemble_nelder_mead2_lb078544.parquet"
base_sub_df = pl.read_parquet(base_sub_path)
base_sub_df.head()

sample_id,ptend_t_0,ptend_t_1,ptend_t_2,ptend_t_3,ptend_t_4,ptend_t_5,ptend_t_6,ptend_t_7,ptend_t_8,ptend_t_9,ptend_t_10,ptend_t_11,ptend_t_12,ptend_t_13,ptend_t_14,ptend_t_15,ptend_t_16,ptend_t_17,ptend_t_18,ptend_t_19,ptend_t_20,ptend_t_21,ptend_t_22,ptend_t_23,ptend_t_24,ptend_t_25,ptend_t_26,ptend_t_27,ptend_t_28,ptend_t_29,ptend_t_30,ptend_t_31,ptend_t_32,ptend_t_33,ptend_t_34,ptend_t_35,…,ptend_v_31,ptend_v_32,ptend_v_33,ptend_v_34,ptend_v_35,ptend_v_36,ptend_v_37,ptend_v_38,ptend_v_39,ptend_v_40,ptend_v_41,ptend_v_42,ptend_v_43,ptend_v_44,ptend_v_45,ptend_v_46,ptend_v_47,ptend_v_48,ptend_v_49,ptend_v_50,ptend_v_51,ptend_v_52,ptend_v_53,ptend_v_54,ptend_v_55,ptend_v_56,ptend_v_57,ptend_v_58,ptend_v_59,cam_out_NETSW,cam_out_FLWDS,cam_out_PRECSC,cam_out_PRECC,cam_out_SOLS,cam_out_SOLL,cam_out_SOLSD,cam_out_SOLLD
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""test_0""",0.000003,-0.000065,-0.000085,-0.000086,-0.000096,-0.0001,-0.000082,-0.000064,-0.000049,-0.000041,-0.000028,-0.000018,-0.000009,-0.000004,-4.1657e-7,0.000001,0.000006,0.000004,0.000003,-0.000009,0.000009,-0.000003,-0.000012,-0.000016,-0.000015,-0.000013,-0.000016,-0.000018,-0.000023,-0.000031,-0.000037,-0.000032,-0.000023,-0.000018,-0.000017,-0.000015,…,9.3603e-7,0.000002,-4.3578e-7,-0.000002,-0.000001,-6.4969e-7,-8.2227e-7,-0.000003,-7.8813e-7,-5.8951e-7,1.9074e-7,-0.000003,0.000006,0.000001,0.000002,-0.000005,-0.000006,-0.000026,-0.000048,-0.000048,-0.000018,0.000038,0.000043,0.000042,0.000022,0.000012,0.000009,0.000008,4.1996e-7,-0.136167,402.35946,4.6684e-11,4.1327e-10,-0.033577,-0.049489,0.09329,0.212398
"""test_10""",-0.000015,-0.00005,-0.000034,-0.000052,-0.00009,-0.000133,-0.000142,-0.000097,-0.000046,-0.000026,-0.000018,-0.000014,-0.000011,-0.000009,-0.000007,-0.000005,-0.000004,-0.000004,-0.000002,-0.000004,-0.000004,-0.000008,-0.000008,-0.000009,-0.000008,-0.000011,-0.000013,-0.000017,-0.00002,-0.000018,-0.00002,-0.000019,-0.000015,-0.000007,-0.000002,0.000003,…,-4.8681e-7,-3.7356e-7,2.0193e-7,4.7321e-7,3.4525e-7,2.1168e-7,1.7050e-7,6.5687e-8,2.0065e-8,2.7567e-9,1.3472e-7,-8.9901e-7,4.5322e-7,-3.2823e-7,0.000001,-4.6198e-9,-5.7225e-7,-0.000002,-0.000002,-0.000002,-0.000002,-0.000001,-1.4282e-7,9.2288e-7,0.000002,0.000001,8.3161e-7,-0.000002,0.000004,-0.027761,345.154827,2.5575e-11,-5.9339e-10,-0.316943,-0.456919,0.0302,0.172592
"""test_100""",-0.000007,-0.00008,-0.000033,-0.000028,-0.000073,-0.000096,-0.000079,-0.000053,-0.00003,-0.000021,-0.000017,-0.000014,-0.000012,-0.00001,-0.000007,-0.000005,-0.000004,-0.000005,0.000004,-0.000009,-0.000004,-0.000007,-0.000006,-0.000006,-0.000004,-0.000003,-0.000006,-0.000006,-0.000007,-0.000011,-0.000015,-0.000019,-0.000022,-0.000022,-0.000024,-0.000024,…,-0.000002,-0.000002,-7.8204e-7,2.2820e-7,8.2554e-7,0.000001,8.8598e-7,8.5306e-7,6.2661e-7,3.6903e-7,1.7289e-8,-4.6920e-7,-1.9907e-7,0.000002,3.3318e-7,-0.000001,-7.2683e-7,-0.000004,-0.00001,-0.00002,-0.000023,0.000009,0.00001,0.000014,0.00001,0.000004,0.000004,0.000004,0.000012,2.065205,331.870707,3.9388e-12,6.2842e-10,0.947471,0.65194,0.288081,0.205645
"""test_1000""",0.000026,0.000007,0.000033,0.000026,-0.000013,-0.000057,-0.000047,-0.000021,-0.000008,-0.000003,-0.000001,-6.6297e-7,-3.7065e-7,0.000002,0.000003,0.000005,0.000006,0.000004,1.3132e-7,7.6130e-7,-4.3587e-7,-0.000002,-0.00001,-0.000019,-0.000017,-0.000014,-0.000014,-0.000017,-0.000021,-0.000023,-0.000021,-0.00002,-0.000018,-0.000017,-0.000016,-0.000017,…,2.6728e-8,-4.6045e-8,6.6996e-7,-0.000001,8.5986e-8,0.000001,-0.000001,1.5021e-7,-9.4691e-7,3.3675e-7,-0.000005,0.000007,0.000003,0.000005,0.000007,0.000007,0.000007,0.00001,0.00001,0.000008,0.000006,0.000004,9.9584e-7,-0.000006,-0.000012,-0.000022,-0.000039,-0.000046,0.000025,2.73912,361.004821,-3.

## timestamp使った予測値で置き換える

In [ ]:
time_sub_path = "gs://kaggle-leap/kami/experiments/610_sim6/mid/submission.parquet"
time_sub_df = pl.read_parquet(time_sub_path)
time_sub_df.head()

In [6]:
# class_exp = "701_multi_class/small"
class_exp = "700_binary_classify/small"

index,0,1
i64,f32,f32
0,0.999998,0.000002
1,0.000003,0.999997
2,0.999998,0.000002
3,0.999993,0.000007
4,0.000003,0.999997


In [23]:
# 閾値ごとの値をチェック

from sklearn.metrics import precision_score, recall_score

val2_predict_df = pl.read_parquet(
    f"gs://kaggle-leap/kami/experiments/{class_exp}/val2_predict.parquet"
)
val2_label_df = pl.read_parquet(
    f"gs://kaggle-leap/kami/experiments/{class_exp}/val2_label.parquet"
)

valid_preds = val2_predict_df.select(["0"]).to_numpy()  # classの番号
valid_y_class = val2_label_df.select(["0"]).to_numpy() ==0

thresholds = [0.5, 0.7, 0.9, 0.99, 0.999, 0.9999, 0.99999]
for threshold in thresholds:
    y_pred = (valid_preds > threshold).astype(np.int64)
    # リコールと精度を計算
    num_cand = np.sum(y_pred)
    precision = precision_score(valid_y_class, y_pred)
    recall = recall_score(valid_y_class, y_pred)
    print(
        f"Threshold: {threshold:.5f}, Num:{num_cand}, Precision: {precision:.5f}, Recall: {recall:.5f}"
    )

Threshold: 0.50000, Num:164917, Precision: 0.98712, Recall: 0.9997
Threshold: 0.70000, Num:164431, Precision: 0.98990, Recall: 0.9996
Threshold: 0.90000, Num:163780, Precision: 0.99341, Recall: 0.9991
Threshold: 0.99000, Num:162670, Precision: 0.99731, Recall: 0.9963
Threshold: 0.99900, Num:160517, Precision: 0.99918, Recall: 0.9849
Threshold: 0.99990, Num:151902, Precision: 0.99993, Recall: 0.9328
Threshold: 0.99999, Num:106162, Precision: 1.00000, Recall: 0.6519


In [25]:
class_pred_df = pl.read_parquet(
    f"gs://kaggle-leap/kami/experiments/{class_exp}/test_predict.parquet"
)
class_preds = val2_predict_df.select(["0"]).to_numpy()
class_pred_df.head()

index,0,1
i64,f32,f32
0,0.466391,0.533609
1,0.451819,0.548181
2,0.455463,0.544537
3,0.380917,0.619083
4,0.454082,0.545918


In [19]:
# 閾値以上の行について、カラムごとにR2を計算して一定値以上であれば置き換えを行う

threshold = 0.99999
replace_index = class_preds > threshold

replace_sub_df = base_sub_df.colne()

for col in base_sub_df.columns[1:]:
    base_pred = base_sub_df[replace_index][col].to_numpy()

0
f64
0.999995
0.999997
0.999996
0.999997
0.999998
…
0.999887
0.999997
0.999991
